In [ ]:
import contextily
import geopandas
import matplotlib.pyplot as plt
import pandas
import partridge
import toolz
from fsspec.implementations.git import GitFileSystem
from mpl_toolkits.axes_grid1 import make_axes_locatable


GTFS_FILE = "gtfs_bus.zip"
ref = "6e08004d9a7603f4411e23da0dac9b0462323a76"

fs = GitFileSystem("gtfs_bus", ref=ref)

with fs.open(GTFS_FILE, "rb") as infile:
    data = infile.read()
with open("/tmp/" + GTFS_FILE, "wb") as outfile:
    outfile.write(data)

In [ ]:
feed = partridge.load_geo_feed("/tmp/" + GTFS_FILE)
service_by_date = partridge.read_service_ids_by_date("/tmp/" + GTFS_FILE)
service_by_date = pandas.DataFrame.from_records(
    toolz.concat([(k, s) for s in v] for k, v in service_by_date.items()),
    columns=["date", "service_id"],
)

In [ ]:
updates = pandas.read_parquet("trip-updates/trip-updates-2022-01-19.parquet")

In [ ]:
date = pandas.Timestamp.today(tz="US/Pacific").date()
service = service_by_date[service_by_date.date == date]
trips = feed.trips[feed.trips.service_id.isin(service.service_id)]
routes = feed.routes.assign(
     route=lambda x: x.route_short_name.combine_first(x.route_long_name)
)[["route_id", "route"]]
trip_timings = (
    feed
    .stop_times
    .groupby("trip_id")
    .agg({"arrival_time": max, "departure_time": min})
)
trip_timings = trip_timings.assign(
    duration=pandas.to_timedelta(
        trip_timings.arrival_time - trip_timings.departure_time,
        unit="s"
    ),
    arrival_time=pandas.to_datetime(date).normalize() + pandas.to_timedelta(trip_timings.arrival_time, unit="s"),
    departure_time=pandas.to_datetime(date).normalize() + pandas.to_timedelta(trip_timings.departure_time, unit="s"),

)
trips = trips.merge(trip_timings, left_on="trip_id", right_index=True)
trips = trips.merge(routes, on="route_id", how="left",)
trips = (
    trips.merge(updates, on="trip_id", how="left")
    .assign(
        schedule_relationship=lambda x: x.schedule_relationship.fillna("scheduled")
    )       
)

In [ ]:
import altair
altair.data_transformers.disable_max_rows()

In [ ]:
altair.Chart(
    trips[["departure_time", "route", "schedule_relationship"]].replace("scheduled", "run"),
    width=1024
).mark_tick(
    size=10,
).encode(
    x=altair.X("departure_time:T", title="Departure Time"),
    y=altair.Y("route:N", title="Route"),
    color=altair.Color(
        'schedule_relationship:N',
        title="Status",
        scale=altair.Scale(domain=["run", "canceled"], range=["darkgreen", "red"])  
    ),
).configure_view(strokeWidth=0)

In [ ]:
geoms = geopandas.GeoDataFrame(
    (
        trips.groupby("route")
        .agg({"shape_id": lambda g: g.value_counts().index[0]})
        .reset_index()
        .merge(feed.shapes, how="left", on="shape_id")
    ),
    crs="EPSG:4326",
    geometry="geometry",
)
trips_agg = (
    trips
    .assign(count=1)   # unused column for count
    .groupby(["route", "schedule_relationship"], dropna=False)
    .agg({
        "duration": sum,
        "count": "count",
    })
    .unstack(level=1)
)
# https://stackoverflow.com/questions/45878333/merge-multiindex-columns-together-into-1-level
trips_agg.columns = ['_'.join(col) for col in trips_agg.columns.values]
trips_agg = trips_agg.assign(
    duration_canceled=trips_agg.duration_canceled.fillna(pandas.Timedelta(0)),
    count_canceled=trips_agg.count_canceled.fillna(0).astype(int),
    count_scheduled=trips_agg.count_scheduled.astype(int),
)
trips_agg = trips_agg.assign(
    percent_duration_canceled=(
        100*trips_agg.duration_canceled/
        (trips_agg.duration_canceled + trips_agg.duration_scheduled)
    ),
    percent_trips_canceled=(
        100*trips_agg.count_canceled/
        (trips_agg.count_canceled + trips_agg.count_scheduled)
    ),
)

gdf = geopandas.GeoDataFrame(
    trips_agg.merge(geoms, on="route"),
    crs="EPSG:4326",
    geometry="geometry"
)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(16,16))
ax.axis("off")
ax.set_title(f"Metro Bus Cancellations for {date.strftime('%A, %B %-m')}")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="2%", pad=0.0)
gdf.to_crs(epsg=3857).plot(
    ax=ax,
    column=gdf.percent_duration_canceled,
    cmap="RdYlGn_r",
    lw=2,
    legend=True,
    cax=cax,
    legend_kwds={"label": "Percent of service hours canceled"}
)
contextily.add_basemap(ax, source=contextily.providers.CartoDB.DarkMatterNoLabels)

In [ ]:
(
    trips_agg
    .sort_values("percent_duration_canceled", ascending=False)
    .reset_index()
    .rename(columns={
        "route": "Route",
        "duration_canceled": "Service Hours Canceled",
        "duration_scheduled": "Service Hours Run",
        "count_canceled": "Trips Canceled",
        "count_scheduled": "Trips Run",
        "percent_duration_canceled": "Percent of Service Hours Canceled",
        "percent_trips_canceled": "Percent of Trips Canceled",
    })
    .style
    .hide_index()
    .format(precision=0)
)

In [ ]:
gdf.duration_canceled.sum()/(gdf.duration_canceled.sum() + gdf.duration_scheduled.sum())

In [ ]:
gdf.count_canceled.sum()/(gdf.count_canceled.sum() + gdf.count_scheduled.sum())